In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import datetime
import csv

C:\Users\jay rana\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('it_tickers.csv')
df
tickers = df['Symbol'].to_numpy()

In [3]:
def extract_tickers_and_info(url, csv_writer):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    print(response)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all elements containing ticker symbols
        ticker_elements = soup.find_all('fin-ticker')
        visited = set()
        for ticker_element in ticker_elements:
            #print(ticker_element)
            ticker = ticker_element.get('symbol')
            if ticker in tickers and ticker not in visited:
                h1_content = soup.find('h1', id='caas-lead-header-undefined')
                if h1_content:
                    h1_text = h1_content.text.strip()
                    print("Heading", h1_text)
                else:
                    print("No h1 tag found with the specified id")
                print("Ticker:", ticker)
                visited.add(ticker)
                # Find the corresponding information
                time_info = soup.find('time').text.strip()
                print(time_info)
                info_element = ticker_element.find_next('div', class_='caas-body')

                if info_element:
                    # Find all <p> tags within the <div> and concatenate them
                    content = ' '.join(paragraph.text.strip() for paragraph in info_element.find_all('p'))
                else:
                    content = "No information found"
                #row = pd.DataFrame([{'time': time_info, 'ticker': ticker, 'heading': h1_text, 'content': content}])
                #df = pd.concat([df, row], ignore_index=True)
                #df.to_csv('E:/news_data.csv', index=False)
                #with open(file_path, 'a', encoding='utf-8') as file:
                #    file.write(f"{time_info},{ticker},{h1_text},{content}\n")
                csv_writer.writerow([time_info, ticker, h1_text, content])

            else:
                print('no data for IT ticekrs')
    else:
        print(f"Failed to retrieve data from {url}")

#df = pd.DataFrame(columns=['time', 'ticker', 'heading', 'content'])
#extract_tickers_and_info('', df)
file_path = 'news_data_2024.csv'


In [4]:
def yahoo_finance_scraper(url,csv_writer):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        list_element = soup.find('ul', class_='Fz(14px) M(0) P(0)')
        
        if list_element:
            list_items = list_element.find_all('li')
            
            for item in list_items:
                link = item.find('a')
                if link:
                    href = link['href']
                    time.sleep(1)
                    #print(href)
                    
                    extract_tickers_and_info(href,csv_writer)
            
            next_link = soup.find('a', class_='C($c-fuji-grey-k)', text='Next')
            if next_link:
                next_url = next_link['href']
                time.sleep(2)
                #print("Scraping next page:", next_url)
                yahoo_finance_scraper(next_url,csv_writer)
                        #csvfile.flush()

        else:
            print("No list items found on the page")
    else:
        print("Failed to retrieve data from Yahoo Finance")

In [5]:
def get_data_for_year(year):
    base_url = "https://finance.yahoo.com/sitemap/"
    start_day =1
    
    #csv_writer.writerow(['time', 'ticker', 'heading', 'content'])
    for month in range(11, 12):
        #if month==9:
        #    start_day=23
        #else:
        #    start_day=1
        for day in range(start_day, 32):
            try:
                date_str = f"{year}_{month:02d}_{day:02d}"
                url = f"{base_url}{date_str}"
                print("Scraping data for date:", date_str)
                with open('news_data_2019_11.csv', 'a', newline='', encoding='utf-8') as csvfile:
                        csv_writer = csv.writer(csvfile)
                        yahoo_finance_scraper(url,csv_writer)
                        csvfile.flush()
            except Exception as e:
                print(f"Error occurred while scraping data for {date_str}: {str(e)}")
get_data_for_year(2019)

Scraping data for date: 2019_11_01
<Response [200]>


KeyboardInterrupt: 